In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
"""def _scrape_price_stats(url: str) -> dict:
    """Scrape price statistics using Selenium with cookie banner handling."""
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Uncomment for headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-notifications") 


    try:
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        driver.set_page_load_timeout(20)

        wait = WebDriverWait(driver, 15)

        try:
            print(f"Accessing URL: {url}")
            driver.get(url)

            # Wait for page load
            print("Waiting for page to load...")
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # Handle cookie banner (if present)
            print("Checking for cookie banner...")
            try:
                cookie_button = wait.until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".onetrust-close-btn-handler"))
                )
                print("Found cookie banner")
                cookie_button.click()
                print("Accepted cookies")
                time.sleep(1)  # Wait for banner to disappear
            except Exception as cookie_error:
                print(f"No cookie banner found or error handling it: {cookie_error}")

            # Add a small delay 
            time.sleep(2)

            print("Looking for statistics section...")
            try:
                # Find the "Have" and "Want" buttons and click "Have"
                have_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'Have')]"))
                )
                have_button.click()
                time.sleep(2)  # Wait for content to load

                # Now find the price elements within the "Have" section
                price_elements = wait.until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".community-stats-section dl"))
                )

                price_info = {'low': None, 'median': None, 'high': None}

                for element in price_elements:
                    try:
                        label = element.find_element(By.TAG_NAME, "dt").text.strip().lower()
                        value = element.find_element(By.TAG_NAME, "dd").text.strip()
                        
                        if 'lowest' in label or 'low' in label:
                            price_info['low'] = value
                        elif 'median' in label:
                            price_info['median'] = value
                        elif 'highest' in label or 'high' in label:
                            price_info['high'] = value

                    except Exception as e:
                        print(f"Error extracting price: {e}")

                print("Final price information:", price_info)
                return price_info

            except Exception as stats_error:
                print(f"Error finding statistics section: {stats_error}")
                return {'low': None, 'median': None, 'high': None}

        except Exception as e:
            print(f"Error during scraping: {e}")
            return {'low': None, 'median': None, 'high': None}

        finally:
            print("Closing browser...")
            driver.quit()

    except Exception as e:
        print(f"Error initializing Chrome driver: {e}")
        return {'low': None, 'median': None, 'high': None}

"""

In [ ]:
def _scrape_price_stats(url: str):
        """Scrape price statistics from Discogs webpage using Selenium."""
        # Set Chrome options (you can enable headless mode if desired)
        chrome_options = Options()
        #chrome_options.add_argument("--headless")
        
        # Initialize WebDriver using WebDriver Manager
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        
        try:
            # Load the Discogs release page
            driver.get(url)
            
            # Wait for the "release-stats" section to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "release-stats"))
            )
            
            # Locate the price statistics section
            price_section = driver.find_element(By.ID, "release-stats")

            # Initialize a dictionary to store price data
            price_info = {'low': None, 'median': None, 'high': None}

            # Locate all <li> elements in the price section containing price stats
            li_elements = price_section.find_elements(By.TAG_NAME, "li")
            
            for li in li_elements:
                # Each <li> should have two spans: one for the label, one for the value
                spans = li.find_elements(By.TAG_NAME, "span")
                if len(spans) == 2:
                    label = spans[0].text.strip().lower()
                    value = spans[1].text.strip()

                    # Map the label to the appropriate field in price_info
                    if 'low' in label:
                        price_info['low'] = value
                    elif 'median' in label:
                        price_info['median'] = value
                    elif 'high' in label:
                        price_info['high'] = value
            
            return price_info

        except Exception as e:
            print(f"Error scraping prices with Selenium: {e}")
            return {'low': None, 'median': None, 'high': None}
        
        finally:
            # Close the Selenium driver
            driver.quit()

# Example usage
if __name__ == "__main__":
    url = "https://www.discogs.com/release/3058849-Floorplan-Sanctified-EP"
    price_data = _scrape_price_stats(url)
    print(price_data)

{'low': '€9.70', 'median': '€15.00', 'high': '€26.74'}
